In [236]:
import numpy as np
from matplotlib import pyplot as plt
import time


from sklearn.svm import SVC
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers

In [237]:
d50=np.random.rand(500,2)
d200=np.random.rand(2000,2)
d500=np.random.rand(5000,2)
d1000=np.random.rand(10000,2)

In [238]:
c50=np.random.choice([-1,1],500).reshape(-1,1)
c200=np.random.choice([-1,1],2000).reshape(-1,1)
c500=np.random.choice([-1,1],5000).reshape(-1,1)
c1000=np.random.choice([-1,1],10000).reshape(-1,1)


In [239]:
# hard-coded SVM
def mySVM(X,y):    
    C = 100
    m,n = X.shape
    Xy = y * X
    H = np.dot(Xy , Xy.T)

    P = cvxopt_matrix(H)
    q = cvxopt_matrix(-np.ones((m, 1)))
    G = cvxopt_matrix(np.vstack((np.eye(m)*-1,np.eye(m))))
    h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * C)))
    A = cvxopt_matrix(y.T*1.0)
    b = cvxopt_matrix(np.zeros(1))
    
    start=time.time()
    sol = cvxopt_solvers.qp(P, q, G, h, A, b)
    end=time.time()

    lambdas = np.array(sol['x'])
    w = ((y * lambdas).T @ X).reshape(-1,1)
    Sv = ((lambdas>0.09999) ).flatten()
    w0 = np.mean(y[Sv] - np.dot(X[Sv], w))
    return((end-start)*10**3, w, w0 )


In [240]:
#sklearn SVM
def skSvm(X,y):
    starts = time.time()
    model = SVC(C = 100, kernel = 'linear')
    model.fit(X, y.reshape(-1)) 
    ends = time.time()
    return((ends-starts)*10**3 ,model.coef_, model.intercept_)


In [241]:
mySVM(d50,c50)

     pcost       dcost       gap    pres   dres
 0: -2.4410e+04 -6.6873e+05  6e+05  1e-13  3e-13
 1: -2.6559e+04 -5.4797e+04  3e+04  6e-14  2e-13
 2: -4.5717e+04 -5.0742e+04  5e+03  9e-13  5e-13
 3: -4.6669e+04 -4.7480e+04  8e+02  4e-13  4e-13
 4: -4.6882e+04 -4.7050e+04  2e+02  2e-13  4e-13
 5: -4.6909e+04 -4.6993e+04  8e+01  2e-13  4e-13
 6: -4.6926e+04 -4.6961e+04  4e+01  1e-13  4e-13
 7: -4.6940e+04 -4.6943e+04  3e+00  6e-14  4e-13
 8: -4.6942e+04 -4.6942e+04  2e-01  6e-14  4e-13
 9: -4.6942e+04 -4.6942e+04  2e-03  5e-13  4e-13
Optimal solution found.


(101.11451148986816,
 array([[-0.25068081],
        [-2.10920014]]),
 1.2729562951547935)

In [242]:
skSvm(d50,c50)

(7.896184921264648, array([[-0.2481787 , -2.11107778]]), array([1.31055879]))

In [ ]:
plt